In [99]:
import csv
import pandas as pd
import os
import scipy.stats
import numpy as np
from datetime import date,timedelta,datetime

In [100]:
def read_data(file):
    df = pd.read_csv(file)
    df = pd.DataFrame(df)
    return df

In [101]:
def mofunc(row):
    if row['Severity'] > 0.8 or row['Hazard_Score'] > 80:
        return 'Warning'
    elif 0.6 < row['Severity'] < 0.80 or 60 < row['Hazard_Score'] < 80:
        return 'Watch'
    elif 0.35 < row['Severity'] < 0.6 or 35 < row['Hazard_Score'] < 60:
        return 'Advisory'
    elif 0 < row['Severity'] < 0.35 or 0 < row['Hazard_Score'] < 35:
        return 'Information'

In [127]:
forcast_date = date.today()
cur_year, cur_month,cur_day = map(str,[forcast_date.today().year,forcast_date.today().month,forcast_date.today().day])
cur_month = cur_month.zfill(2)
cur_day=cur_day.zfill(2)
#MOMOutput='Final_Attributes_'+cur_year+cur_month+str(int(cur_day)-1)+'18.csv'
MOMOutput='Final_Attributes_20210621.csv'
VIIRS="VIIRS_Flood_"+cur_year+cur_month+str(int(cur_day)-4)+'.csv'


In [128]:
weightage = read_data('VIIRS_Weightages.csv')
Attributes=read_data('Attributes.csv')
PDC_resilience = read_data('Copy of Resilience_Index.csv')
add_field_VIIRS=['VIIRS_area_1day_score', 'VIIRS_percarea_1day_score', 'VIIRS_area_5day_score', 'VIIRS_percarea_5day_score','VIIRSTotal_Score']

In [129]:
#Read VIIRS Processing data and calculate score
with open(VIIRS, 'r', encoding='UTF-8') as VIIRS_file:
    VIIRS_reader = csv.reader(VIIRS_file)
    csvfile = open('VIIRS_w_score.csv', 'w', newline='\n', encoding='utf-8')
    VIIRS_w_score = csv.writer(csvfile)
    row_count = 1
    # csv_writer = csv.writer(write_obj)
    for row in VIIRS_reader:
        if row_count == 1:
            for x in add_field_VIIRS:
                row.append(x)
            row_count = row_count + 1
        else:
            if float(row[1]) / float(weightage.VIIRS_Area_wt) > float(weightage.VIIRS_Area_max_pt):
                VIIRS_area_1day_score = str(float(weightage.VIIRS_Area_max_pt)*float(weightage.one_Day_Multiplier))
            else:
                VIIRS_area_1day_score = str(float(weightage.VIIRS_Area_Min_pt) * float(weightage.one_Day_Multiplier)* float(row[1]) / float(weightage.VIIRS_Area_wt))
            if float(row[2]) / float(weightage.VIIRS_percArea_wt) > float(weightage.VIIRS_percArea_Maxpt):
                VIIRS_perc_area_1day_score = str(float(weightage.VIIRS_percArea_Maxpt)*float(weightage.one_Day_Multiplier))
            else:
                VIIRS_perc_area_1day_score = str(float(weightage.VIIRS_percArea_Minpt)*float(weightage.one_Day_Multiplier)* float(row[2]) / float(weightage.VIIRS_percArea_wt))
            if float(row[3]) / float(weightage.VIIRS_Area_wt) > float(weightage.VIIRS_Area_max_pt):
                VIIRS_area_5day_score = str(float(weightage.VIIRS_Area_max_pt)*float(weightage.five_Day_Multiplier))
            else:
                VIIRS_area_5day_score = str(float(weightage.VIIRS_Area_Min_pt) * float(weightage.five_Day_Multiplier)* float(row[3]) / float(weightage.VIIRS_Area_wt))
            if float(row[4]) / float(weightage.VIIRS_percArea_wt) > float(weightage.VIIRS_percArea_Maxpt):
                VIIRS_perc_area_5day_score = str(float(weightage.VIIRS_percArea_Maxpt)*float(weightage.five_Day_Multiplier))
            else:
                VIIRS_perc_area_5day_score = str(float(weightage.VIIRS_percArea_Minpt)*float(weightage.five_Day_Multiplier)* float(row[4]) / float(weightage.VIIRS_percArea_wt))          
            Sum_Score = str(
                (float(VIIRS_area_1day_score) + float(VIIRS_perc_area_1day_score) + float(VIIRS_area_5day_score) + float(VIIRS_perc_area_5day_score)))
            score_field = [VIIRS_area_1day_score, VIIRS_perc_area_1day_score, VIIRS_area_5day_score, VIIRS_perc_area_5day_score, Sum_Score]
            for x in score_field:
                row.append(x)
        VIIRS_w_score.writerow(row)
csvfile.close()

In [ ]:
VIIRS = read_data('VIIRS_w_score.csv')
VIIRS = VIIRS[VIIRS.VIIRSTotal_Score > 0.1]
MOM = read_data(MOMOutput)
MOM.drop(columns=['area_km2','ISO','Admin0','Admin1','rfr_score','cfr_score','Resilience_Index',' NormalizedLackofResilience ','Severity','Alert'], inplace=True)
Final_Output_0= pd.merge(MOM.set_index('pfaf_id'), VIIRS.set_index('pfaf_id'), on='pfaf_id', how='outer')
join1 = pd.merge(Attributes, PDC_resilience[['ISO', 'Resilience_Index', ' NormalizedLackofResilience ']], on='ISO', how='inner')
Final_Output=pd.merge(join1.set_index('pfaf_id'), Final_Output_0, on='pfaf_id', how='right')
Final_Output[['Hazard_Score']] = Final_Output[['Hazard_Score']].fillna(value=0)
Final_Output.loc[(Final_Output['Hazard_Score']!=0) & (Final_Output['Hazard_Score']<Final_Output['VIIRSTotal_Score']),'Flag']=3
Final_Output['Hazard_Score'] =Final_Output[['Hazard_Score', 'VIIRSTotal_Score']].max(axis=1)
Final_Output = Final_Output[Final_Output.Hazard_Score != 0]
Final_Output = Final_Output.assign(
    Scaled_Riverine_Risk=lambda x: Final_Output['rfr_score'] * 20)
Final_Output = Final_Output.assign(
    Scaled_Coastal_Risk=lambda x: Final_Output['cfr_score'] * 20)
Final_Output = Final_Output.assign(
    MoM_Severity=lambda x: scipy.stats.norm(np.log(100 - Final_Output[['Scaled_Riverine_Risk', 'Scaled_Coastal_Risk']].max(axis=1)), 1).cdf(
        np.log(Final_Output['Hazard_Score'])))
Final_Output = Final_Output.assign(
    VIIRS_Severity=lambda x: scipy.stats.norm(np.log(100 - Final_Output[['Scaled_Riverine_Risk', 'Scaled_Coastal_Risk']].max(axis=1)), 1).cdf(
        np.log(Final_Output['VIIRSTotal_Score'])))
Final_Output['Alert'] = Final_Output.apply(mofunc, axis=1)
Final_Output.to_csv('Final_Attributes_'+cur_year+cur_month+str(float(cur_day)-1)+'18_VIIRSUpdated.csv', encoding='utf-8-sig')
join1 = pd.merge(Attributes, PDC_resilience[['ISO', 'Resilience_Index', ' NormalizedLackofResilience ']], on='ISO', how='inner')
Attributes_Clean_VIIRS_Updated = pd.merge(join1.set_index('pfaf_id'), Final_Output[['Alert','Flag']], on='pfaf_id', how='right')
Attributes_Clean_VIIRS_Updated.to_csv('Attributes_Clean'+cur_year+cur_month+str(float(cur_day)-1)+'18_VIIRSUpdated.csv', encoding='utf-8-sig')
#os.remove('VIIRS_w_score.csv')

In [130]:
def mofunc0(row):
    if row['MoM_Severity'] > 0.8 or row['Hazard_Score'] > 80:
        return 'Warning'
    elif 0.6 < row['MoM_Severity'] < 0.80 or 60 < row['Hazard_Score'] < 80:
        return 'Watch'
    elif 0.35 < row['MoM_Severity'] < 0.6 or 35 < row['Hazard_Score'] < 60:
        return 'Advisory'
    elif 0 < row['MoM_Severity'] < 0.35 or 0 < row['Hazard_Score'] < 35:
        return 'Information'

In [131]:
def mofunc1(row):
    if row['VIIRS_Severity'] > 0.8 or row['VIIRSTotal_Score'] > 80:
        return 'Warning'
    elif 0.6 < row['VIIRS_Severity'] < 0.80 or 60 < row['VIIRSTotal_Score'] < 80:
        return 'Watch'
    elif 0.35 < row['VIIRS_Severity'] < 0.6 or 35 < row['VIIRSTotal_Score'] < 60:
        return 'Advisory'
    elif 0 < row['VIIRS_Severity'] < 0.35 or 0 < row['VIIRSTotal_Score'] < 35:
        return 'Information'

In [132]:
## To get the confusion matrix i.e. keeping MOMalert as well as VIIRS Alert
VIIRS = read_data('VIIRS_w_score.csv')
VIIRS = VIIRS[VIIRS.VIIRSTotal_Score > 0.1]
MOM = read_data(MOMOutput)
#MOM.drop(columns=['area_km2','ISO','Admin0','Admin1','rfr_score','cfr_score','Resilience_Index',' NormalizedLackofResilience ','Severity','Alert'], inplace=True)
MOM.drop(columns=['ISO','Admin0','Admin1','rfr_score','cfr_score','Resilience_Index',' NormalizedLackofResilience ','Severity','Alert'], inplace=True)
Final_Output_0= pd.merge(MOM.set_index('pfaf_id'), VIIRS.set_index('pfaf_id'), on='pfaf_id', how='outer')
join1 = pd.merge(Attributes, PDC_resilience[['ISO', 'Resilience_Index', ' NormalizedLackofResilience ']], on='ISO', how='inner')
Final_Output=pd.merge(join1.set_index('pfaf_id'), Final_Output_0, on='pfaf_id', how='right')
Final_Output[['Hazard_Score']] = Final_Output[['Hazard_Score']].fillna(value=0)
Final_Output.loc[(Final_Output['Hazard_Score']!=0) & (Final_Output['Hazard_Score']<Final_Output['VIIRSTotal_Score']),'Flag']=3
#Final_Output['Hazard_Score'] =Final_Output[['Hazard_Score', 'VIIRSTotal_Score']].max(axis=1)
#Final_Output = Final_Output[Final_Output.Hazard_Score != 0]
Final_Output = Final_Output.assign(
    Scaled_Riverine_Risk=lambda x: Final_Output['rfr_score'] * 20)
Final_Output = Final_Output.assign(
    Scaled_Coastal_Risk=lambda x: Final_Output['cfr_score'] * 20)
Final_Output = Final_Output.assign(
    MoM_Severity=lambda x: scipy.stats.norm(np.log(100 - Final_Output[['Scaled_Riverine_Risk', 'Scaled_Coastal_Risk']].max(axis=1)), 1).cdf(
        np.log(Final_Output['Hazard_Score'])))
Final_Output = Final_Output.assign(
    VIIRS_Severity=lambda x: scipy.stats.norm(np.log(100 - Final_Output[['Scaled_Riverine_Risk', 'Scaled_Coastal_Risk']].max(axis=1)), 1).cdf(
        np.log(Final_Output['VIIRSTotal_Score'])))
Final_Output['MoM_Alert'] = Final_Output.apply(mofunc0, axis=1)
Final_Output['VIIRS_Alert'] = Final_Output.apply(mofunc1, axis=1)
Final_Output.to_csv('Final_Attributes_MOM+VIIRSAlert'+cur_year+cur_month+str(float(cur_day)-4)+'18_VIIRSUpdated.csv', encoding='utf-8-sig')
#join1 = pd.merge(Attributes, PDC_resilience[['ISO', 'Resilience_Index', ' NormalizedLackofResilience ']], on='ISO', how='inner')
#Attributes_Clean_VIIRS_Updated = pd.merge(join1.set_index('pfaf_id'), Final_Output[['Alert','Flag']], on='pfaf_id', how='right')
#Attributes_Clean_VIIRS_Updated.to_csv('Attributes_Clean'+cur_year+cur_month+str(float(cur_day)-1)+'18_VIIRSUpdated.csv', encoding='utf-8-sig')
#os.remove('VIIRS_w_score.csv')